# Chat Completion API

In [ ]:
!pip install openai # 로컬 작업 시 설

## 환경설정
1. openai 사이트에서 API KEY 발급
2. google colab - secret 탭에 `OPENAI_API_KEY` 등록

설정 -> API Keys

In [4]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [5]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "너는 아주 친절하고, 많은 도움을 주는 챗봇이야~"
        }
      ]
    },
    {
      "role": "user", # 이 부분을 추가하면 대화가 쌓임
      "content": [
        {
          "type": "text",
          "text": "안녕 내 이름은 차은우야"
        }
      ]
    },
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [ ]:
# ChatCompletion(choices=[Choice(message(content))])
response.choices[0].message.content

'안녕 차은우! 만나서 반가워! 어떻게 도와줄 수 있을까?'

## 프롬프팅의 기본구성

https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/

1. Instruction 지시사항
2. Context 문맥
3. Input Data/Example 입력/예시
4. Output Indicator 출력지시

## 기사 제목 교정
- 기자들이 송고한 기사에서 제목을 추출하고, 표현 조정

- 프랑스AFP 속보시스템에서 도입되어 사용

In [ ]:
title_before = '테이의 FM 개꿀 라디오 방송에 주목해주세요.'
title_before = '졸라 빡센 작업으로 끼니 거르기 일쑤인 노동자들의 애환'

# 역할/페르소나/지시사항
system_message = """
기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조등에 있어서 교정작업을 수행해 주세요.

- 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
- 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
- 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
- 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고, 의미를 적절히 유지하도록 제목을 교정하세요.

### Steps ###
1. 기사제목을 읽고 주요내용을 이해하세요.
2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
3. 맞춤법, 문법, 의미 전달의 정확성등을 점검하고 적절히 수정하세요.
4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈수 있는지 점검하고 매우 간결하게 정리하세요.


### Output Format ###
기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

- 원래 제목: [기사 원래 제목]
- 교정 제목: [교정 기사 제목]

### Examples ###
- 원래 제목: "어제 서울에서 큰 불이 나 수백명이 대피했다."
- 교정 제목: "서울 대형 화재, 수백명 대피"

- 원래 제목: "전기자동차 판매량 급감에 내연차회사들이 즐거워하는 중입니다."
- 교정 제목: "전기차 판매량 급감에 웃는 내연차회사들"

### Extra Instructions ###
- 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
- 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
- 제목이 특정집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

"""

user_message = f"""
다음 기사제목을 교정해주세요.

제목: {title_before}
"""

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": system_message
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": user_message
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)
print(response.choices[0].message.content)

- 원래 제목: 졸라 빡센 작업으로 끼니 거르기 일쑤인 노동자들의 애환
- 교정 제목: 과중한 업무로 끼니를 거르는 노동자들의 애환


In [ ]:
# 함수 리팩토링

def correct_news_title(title_before, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조등에 있어서 교정작업을 수행해 주세요.

    - 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
    - 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
    - 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
    - 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고, 의미를 적절히 유지하도록 제목을 교정하세요.

    ### Steps ###
    1. 기사제목을 읽고 주요내용을 이해하세요.
    2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
    3. 맞춤법, 문법, 의미 전달의 정확성등을 점검하고 적절히 수정하세요.
    4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈수 있는지 점검하고 매우 간결하게 정리하세요.


    ### Output Format ###
    기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

    - 원래 제목: [기사 원래 제목]
    - 교정 제목: [교정 기사 제목]

    ### Examples ###
    - 원래 제목: "어제 서울에서 큰 불이 나 수백명이 대피했다."
    - 교정 제목: "서울 대형 화재, 수백명 대피"

    - 원래 제목: "전기자동차 판매량 급감에 내연차회사들이 즐거워하는 중입니다."
    - 교정 제목: "전기차 판매량 급감에 웃는 내연차회사들"

    ### Extra Instructions ###
    - 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
    - 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
    - 제목이 특정집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

    """

    user_message = f"""
    다음 기사제목을 교정해주세요.

    제목: {title_before}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
                },
                {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
                }
            ],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content

title_before = '주말 미친 폭우 예상, 모두들 무사하시길~'
output = correct_news_title(title_before)
print(output)

- 원래 제목: 주말 미친 폭우 예상, 모두들 무사하시길~
- 교정 제목: "주말 강한 폭우 예보, 모두 안전하세요"


## 연애코치 ReAct

In [10]:
def dating_coach(prompt, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    << System Instruction >>
    어떤 상황에서든 최고의 논리적/감성적 관점을 적용하는 연애코치로써 사용자의 고민을 해결해 주세요.

    << Output Format >>
    1. 상황분석:

    2. 행동계획:

    3. 실행:
    """

    user_message = f"""
    사용자 현재상황:
    {prompt}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content


In [11]:
print(dating_coach('저 이번주말에 여자친구와 100일인데, 기억에 남을 선물을 하고 싶어 뭐가 좋을까?'))

1. 상황분석:
   - 당신은 여자친구와의 100일을 축하하고 싶어하고, 특별한 선물을 통해 기억에 남는 순간을 만들고자 합니다.
   - 여자친구가 어떤 것에 감성적으로 반응하는지, 개인적인 취향은 어떤지를 고려해야 합니다. 예를 들어, 그녀의 관심사, 취미, 그리고 두 사람의 특별한 추억이 있는 물건 등을 생각해볼 필요가 있습니다.
   - 선물은 물질적인 것 뿐만 아니라 경험적인 요소도 중요하므로, 특별한 순간을 함께 만들어 줄 수 있는 방법도 고려해야 합니다.

2. 행동계획:
   - 여자친구의 취향과 관심사를 파악합니다. 그녀가 좋아하는 색, 영화, 음악, 취미 등을 생각해보세요.
   - 선물의 종류를 선택할 때, 다음처럼 여러 가지 옵션을 모색해봅니다:
     1. 개인화 된 선물 (예: 사진 앨범, 맞춤형 쥬얼리, 손 편지)
     2. 특별한 경험 (예: 로맨틱한 저녁 식사, 주말 여행, 스페셜 데이트)
     3. 그녀의 관심사에 맞춘 아이템 (예: 책, 화장품, 악세서리)
   - 선물과 함께 뜻깊은 메세지를 준비해 감정을 표현하세요.

3. 실행:
   - 여자친구가 좋아할만한 선물 아이디어를 구체화합니다. 예를 들어, 두 사람의 추억이 담긴 사진으로 구성한 앨범을 만들거나, 그녀가 좋아하는 장소에서의 소중한 데이트 계획을 세워봅니다.
   - 선물과 관련된 문구나 편지를 작성하여 그녀에게 그 의미를 담아 전달합니다.
   - 그녀와의 100일 기념일에는 특별한 날이 될 수 있도록 분위기를 신경 써서 데이트를 진행하세요. 조명이나 음악, 식사 등을 준비해 특별한 순간을 만들어주세요. 

이렇게 하면 은근하면서도 진심이 담긴 기억에 남는 100일을 만들 수 있을 것입니다!


In [12]:
print(dating_coach('제가 어제 친구들이랑 놀다가 여자친구랑 연락을 잘 못해서, 단단히 삐져있는 상황입니다. 이 위기를 어떻게 극복해야 할까요?'))

1. 상황분석:
   - 여러분의 여자친구는 어제 연락이 잘 되지 않아 불안함이나 서운함을 느꼈을 것입니다. 이러한 감정은 관계에서 불일치를 초래할 수 있으며, 특히 기본적으로 소통이 중요한 연애 관계에서 더욱 그렇습니다. 친구들과의 시간을 중요시하더라도, 여자친구가 느낀 감정을 존중하고 이해하는 것이 중요합니다.

2. 행동계획:
   - 먼저 여자친구에게 사과의 편지를 보내거나 전화로 먼저 연락을 해보세요. “어제 연락이 잘 안 돼서 미안해”라는 문구로 시작해 그녀의 기분을 이해하고 있다는 것을 보여주세요.
   - 그녀와 진솔한 대화를 나누어 보세요. 그녀가 느꼈던 감정과 생각을 받고, 여러분의 상황도 함께 설명하되, 그 상황이 그녀에게 상처가 되었음을 인식하세요.
   - 만약 조금 시간이 필요하다고 느낀다면, 개인적인 공간을 주되, 신경 쓰고 있다는 것을 잊지 않도록 자주 메시지를 보내 손쉽게 좋은 감정을 쌓아가세요.

3. 실행:
   - 오늘 저녁이나 내일 아침 중으로 그녀에게 직접 전화를 걸어보세요. “어제 연락이 잘 안 돼서 미안해. 너와 함께 하는 것이 나에게 너무 중요해”라는 느낌의 말로 시작하세요.
   - 그녀의 반응이 어떻든지 경청하고, 그녀가 하고 싶어하는 대화를 따라가는 것도 좋습니다.
   - 마지막으로, 함께할 수 있는 시간을 제안해 보세요. “이번 주말에 너와 함께 영화 보러 갈까?”와 같은 긍정적인 제안으로 관계 회복을 위한 작은 노력을 보여주세요.

이 모든 과정에서 중요한 것은 진정성과 소통입니다. 그녀가 느낀 감정을 받아들이고 이해해 주는 태도를 지속적으로 보여주세요.


## 냉털마스터 ReAct
- 사용자가 냉장고에 남아있는 음식재료를 알려주면, LLM은 이를 바탕으로 어떤 음식을 만들지를 조언
- Reasoning/Action을 끌어낼 수 있는 적절한 프롬프팅을 작성하세요.

In [17]:
def fridge_raid_master(user_foods, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
당신은 사용자 냉장고의 재료를 가지고 최고의 음식을 만들 수 있는 레시피를 추천하는 챗봇입니다.
현재 상황을 분석하고 실행계획을 세우며 추가 내용이 있을지 확인하는 꼼꼼함을 보여주세요.

## 지시사항

### 상황분석
1. **재료 확인**: 냉장고에 있는 재료 목록을 작성하고, 각 재료의 신선함과 사용 가능 시간을 파악합니다.
2. **요리 컨셉 설정**: 재료를 고려하여 요리의 주제를 정하고 잠재적인 요리 아이디어를 계획합니다.

### 실행계획
3. **목표 변경**: 사용 가능한 재료로 만들고자 하는 요리의 목표를 설정하고, 필요한 정보나 추가 자료를 조사합니다.
4. **전략 개발**: 단계별로 목표를 달성하기 위한 전반적인 요리 계획을 세웁니다.
    단, 사용자가 따라하기 쉽게 단계별 가이드를 작성해야 합니다.

### 검증 및 추가내용 확인
5. **레시피 검토**: 작성된 레시피를 검토하여 정보가 명확하고 완전한지 확인합니다.
6. **수정**: 필요 시 수정하여 최종 레시피를 완성합니다.

## 출력형식

1. 상황분석:
   - 재료 목록 및 상태
   - 요리 컨셉 및 아이디어

2. 실행계획:
   - 목표 및 세부 계획
   - 필요한 추가 자료 및 전략

3. 검증 및 추가내용 확인
   - 레시피의 정확성 검토
   - 수정 및 편집

4. 최종레시피
   - 사용자가 따라하기 쉽게 목록으로 작성
   - 필요한 준ㅂ

## Examples

- **Input**:
    사용자의 냉장고에는 현재 [무, 파, 두부]이/가 있습니다.
-   **Output**:
  1. 상황분석: 오늘은 냉장고에 있는 무, 파, 두부를 이용해 한식 스프를 만들어보겠습니다...
  2. 실행계획: 먼저 무와 파를 얇게 썰어 냄비에 넣고 물을 부어...
  3. 검증 및 추가내용 확인: 레시피를 확인한 결과, 무와 두부의 사용 방법에 대한 설명이 추가로 필요합니다...

## Notes

- 요리의 문화적 관련성과 감수성을 고려합니다.
- 레시피는 예상 독자의 요리 지식 수준에 맞게 조정됩니다.

    """

    user_message = f"""
    사용자의 냉장고에는 현재 {user_foods}이/가 있습니다.
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content

In [18]:
print(fridge_raid_master('두부, 김치, 오이, 가지'))

1. **상황분석:**
   - **재료 목록 및 상태**:
     - 두부: 신선하고 사용 가능
     - 김치: 잘 익었고 사용 가능
     - 오이: 아삭한 상태로 신선함 유지
     - 가지: 약간 물러졌으나 사용 가능

   - **요리 컨셉 및 아이디어**: 오늘은 냉장고에 있는 재료들을 활용하여 한국식 볶음밥이나 두부 김치 볶음을 만들어보는 것을 제안합니다. 두부와 김치를 주 재료로 하여 간단하면서도 맛있는 한 끼를 준비할 수 있습니다. 여기에 오이와 가지를 곁들여 신선한 맛을 추가할 수 있습니다.

2. **실행계획:**
   - **목표 및 세부 계획**:
     - 주 요리: 두부 김치 볶음
     - 부 요리: 오이 가지 볶음
     
     1. 두부는 깨끗이 씻고 물기를 제거한 후, 네모난 모양으로 썰기.
     2. 김치는 작게 썰고, 오이는 반으로 갈라 얇게 썰기.
     3. 가지는 잘라서 소금물에 잠시 담가 두었다가 물기를 제거하기.

   - **필요한 추가 자료 및 전략**:
     - 볶음밥의 조리법에 대해 간단한 조리 시간을 조사
     - 필요한 양념(예: 간장, 고춧가루) 확인하기

3. **검증 및 추가내용 확인**:
   - 레시피 검토 결과, 두부 김치 볶음에 대한 조리 시간 및 방법에 대한 명확한 설명이 필요합니다. 또한, 오이와 가지 볶음을 준비 위한 추가 양념이나 조리 방법을 세부적으로 작성할 필요가 있습니다.

4. **최종레시피**:
   - **재료**:
     - 두부 1모
     - 김치 1컵
     - 오이 1개
     - 가지 1개
     - 식용유 2큰술
     - 간장 1큰술
     - 고춧가루 1작은술 (선택적)
     - 소금과 후추 (기호에 따라)

   - **조리 단계**:
     1. 두부를 네모나게 썰어 물기를 제거합니다.
     2. 중불로 달군 팬에 식용유를 두르고 두부를 넣어 앞뒤로 노릇하게 굽습니다.
     3. 두부가 노릇해지면 김

In [23]:
from IPython.display import Markdown

user_foods = ['소고기', '양파', '표고버섯', '명란젓', '된장']
display(Markdown(fridge_raid_master(user_foods)))

1. 상황분석:
   - **재료 목록 및 상태**: 
     - 소고기: 신선할 경우 사용 가능
     - 양파: 신선하며 사용 가능
     - 표고버섯: 신선하며 사용 가능
     - 명란젓: 개봉 후 사용 기간이 지났다면 확인 필요, 그렇지 않다면 사용 가능
     - 된장: 장기 보관 가능, 사용 가능
     
   - **요리 컨셉 및 아이디어**: 
     오늘의 요리는 '된장찌개'입니다. 이 재료들을 활용하여 깊고 구수한 맛의 된장찌개를 만들 수 있습니다. 소고기는 육수를 내고, 양파와 표고버섯은 식감과 맛을 더해줄 것입니다. 명란젓은 찌개 위에 올려서 풍미를 더할 예정입니다.

2. 실행계획:
   - **목표 및 세부 계획**: 
     소고기, 양파, 표고버섯을 주요 재료로 하여 깊고 풍미 있는 된장찌개를 만드는 것을 목표로 합니다. 명란젓은 마지막에 올려 풍미를 더합니다. 
     
   - **필요한 추가 자료 및 전략**: 
     - 물의 양 조절: 찌개에 필요한 액체의 양을 정하기
     - 조리 도구 준비: 냄비 및 국자
     
3. 검증 및 추가내용 확인:
   - **레시피의 정확성 검토**: 재료의 비율과 간단한 조리법이 포함되어야 합니다. 명란젓 사용법에 대한 설명이 필요하며, 찌개의 끓이는 시간을 추가해야 합니다.
   - **수정 및 편집**: 명란젓을 넣는 방법과 찌개를 먹는 방법에 대한 설명을 추가합니다.

4. 최종레시피:
   - **필요한 재료**:
     - 소고기 200g
     - 양파 1개
     - 표고버섯 100g
     - 된장 2 큰술
     - 명란젓 적당량
     - 물 4컵
     
   - **조리 방법**:
     1. 소고기를 한 입 크기로 썰고, 양파는 채 썰고, 표고버섯은 슬라이스 합니다.
     2. 냄비에 물 4컵을 넣고 소고기를 넣어 끓입니다.
     3. 소고기가 익어갈 때 양파와 표고버섯을 넣고, 된장을 풀어줍니다.
     4. 약한 불에서 15분 정도 끓입니다.
     5. 마지막에 명란젓을 위에 올려서 맛을 더하고, 한 번 더 끓입니다.
     6. 그릇에 담아 따뜻하게 서빙합니다.

이렇게 완성된 된장찌개는 풍부한 맛과 함께 밥과 함께 즐기기 좋은 메뉴입니다. 즐거운 요리 시간 되세요!

## 면접질문 생성 JSON 출력

In [30]:
def job_interview(job_posting, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
당신은 머신러닝/딥러닝/LLM/AI서비스개발의 전문가로써, 해당분야의 으뜸가는 면접관입니다.
매번 그룹사의 인재를 발굴하기 위해 면접질문/모범답안을 작성하고 있습니다.

<<지시사항>>
- 사용자가 제출한 채용공고의 내용을 바탕으로 예상면접질문과 답변을 작성해주세요.
- 하드스킬과 소프트스킬 두개의 섹션으로 나누어 작성해주세요.
- 각 스킬별로 질문/답변을 3개씩 만들어주세요.

<<출력형식>>
출력은 json형식으로만 반환되어야 합니다.

{{
    "hard_skill": [
        {{
            "question": "",
            "answer": "",
        }}
    ],
    "soft_skill": [
        {{
            "question": "",
            "answer": "",
        }}
    ]
}}
"""

    user_message = f"""
채용공고:
{job_posting}
"""

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "json_object"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content

In [33]:
job_posting = """
| 모집부문                      | 담당업무                                                                                                                                                                                                                                                                                               | 자격요건 및 필수사항                                                                                                                                                                                                                                                                                                                                                                 |
| ------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **AI 인프라 구축 (ML/DEVOps)** | 1. AI 모델의 배포를 위한 인프라 구축 및 운영<br>2. 알고리즘 개발자에 대한 기술 지원<br>3. 머신러닝 인프라의 설계 및 개발, 운영<br>4. 모니터링, 로깅 시스템의 개발 및 구현<br>5. 머신러닝 시스템의 학습, 배포, 테스트 환경 구축, 운영 및 자동화<br>6. 오픈소스 기반의 플랫폼 통합 및 내재화 구축<br>(MLflow, LakeFS, CVAT, BentoML 등)<br>7. 보안, 인증, 권한 관리 등 운영 정책 및 관리자 기능 구축<br>8. 시스템과 프로세스에 대한 문서 작업 | **학력**<br>- 대졸이상(4년제)<br><br>**경력**<br>- 신입<br><br>**필수사항**<br>- Python 프로그래밍 능력<br>- 컨테이너 오케스트레이션 툴 활용 능력(Docker, Kubernetes 등)<br>- Jenkins, ArgoCD, Gitlab 등 구축 및 운영 경험<br><br>**우대사항**<br>- DevOps 환경 구축 혹은 DevOps 환경에서 개발 및 배포 경험 보유자<br>- MLflow, Kubeflow, ML 모델 서빙 플랫폼 등과 관련된 실무 경험 보유자<br>- Kubernetes, Docker, GitOps, CI/CD, Vault, MinIO 등 클라우드 네이티브 인프라 경험 보유자 |
"""
output = job_interview(job_posting)
print(type(output))
print(output)

<class 'str'>
{
    "hard_skill": [
        {
            "question": "Python을 활용해 머신러닝 인프라를 구축하는 과정에서 겪었던 가장 어려운 점은 무엇이었나요? 그리고 어떻게 해결하셨나요?",
            "answer": "머신러닝 인프라 구축 중 데이터 처리 및 모델 배포 과정에서 성능 최적화가 큰 도전이었습니다. 특정 데이터 세트를 처리하는 데 오래 걸리는 문제가 있었는데, 이를 해결하기 위해 데이터 전처리 작업을 병렬화하고, 메모리 관리와 캐싱 기법을 도입했습니다. 그 결과, 전체 파이프라인의 처리 속도가 현저히 개선되었습니다."
        },
        {
            "question": "Docker와 Kubernetes를 이용한 컨테이너 오케스트레이션 경험에 대해 구체적으로 설명해주시겠어요?",
            "answer": "이전 프로젝트에서 Docker를 사용해 모델을 컨테이너화하고, Kubernetes를 통한 자동 배포 및 스케일링을 구현했습니다. Kubernetes 클러스터를 설정한 후, Helm 차트를 사용하여 애플리케이션을 쉽게 배포하고 관리했습니다. 또한, 리소스 모니터링 툴을 사용해 성능을 지속적으로 개선하여 안정성을 확보했습니다."
        },
        {
            "question": "CI/CD 파이프라인을 Jenkins와 ArgoCD로 구축한 경험에 대해 상세히 설명해주세요.",
            "answer": "Jenkins를 사용하여 초기 빌드를 자동화하고, 테스트가 통과하면 ArgoCD를 통해 Kubernetes 클러스터에 자동 배포하도록 설정했습니다. 이 과정을 통해 코드 변경이 있을 때마다 자동으로 배포되도록 하여 배포 주기를 단축시키고 개발자의 피드백 시간을 줄일 수 있었습니다. 이 과정에서 발생하는 오류를 빠르게 처리할 수 있는 모니터링 및 알림 시스템도 구축했습니다."
       

In [34]:
import json

# 응답데이터가 json 형식이 아니라면 오류 발생!
data = json.loads(output)
print(type(data))

hard_skill_qa = data['hard_skill']
soft_skill_qa = data['soft_skill']
print(hard_skill_qa)
print(soft_skill_qa)

<class 'dict'>
[{'question': 'Python을 활용해 머신러닝 인프라를 구축하는 과정에서 겪었던 가장 어려운 점은 무엇이었나요? 그리고 어떻게 해결하셨나요?', 'answer': '머신러닝 인프라 구축 중 데이터 처리 및 모델 배포 과정에서 성능 최적화가 큰 도전이었습니다. 특정 데이터 세트를 처리하는 데 오래 걸리는 문제가 있었는데, 이를 해결하기 위해 데이터 전처리 작업을 병렬화하고, 메모리 관리와 캐싱 기법을 도입했습니다. 그 결과, 전체 파이프라인의 처리 속도가 현저히 개선되었습니다.'}, {'question': 'Docker와 Kubernetes를 이용한 컨테이너 오케스트레이션 경험에 대해 구체적으로 설명해주시겠어요?', 'answer': '이전 프로젝트에서 Docker를 사용해 모델을 컨테이너화하고, Kubernetes를 통한 자동 배포 및 스케일링을 구현했습니다. Kubernetes 클러스터를 설정한 후, Helm 차트를 사용하여 애플리케이션을 쉽게 배포하고 관리했습니다. 또한, 리소스 모니터링 툴을 사용해 성능을 지속적으로 개선하여 안정성을 확보했습니다.'}, {'question': 'CI/CD 파이프라인을 Jenkins와 ArgoCD로 구축한 경험에 대해 상세히 설명해주세요.', 'answer': 'Jenkins를 사용하여 초기 빌드를 자동화하고, 테스트가 통과하면 ArgoCD를 통해 Kubernetes 클러스터에 자동 배포하도록 설정했습니다. 이 과정을 통해 코드 변경이 있을 때마다 자동으로 배포되도록 하여 배포 주기를 단축시키고 개발자의 피드백 시간을 줄일 수 있었습니다. 이 과정에서 발생하는 오류를 빠르게 처리할 수 있는 모니터링 및 알림 시스템도 구축했습니다.'}]
[{'question': '알고리즘 개발자와의 협업 과정에서 어떻게 기술 지원을 하셨나요?', 'answer': '개발자들이 인프라를 원활하게 사용할 수 있도록 필요한 툴과 환경을 설정했습니다. 또한 정기적인 소통을 통해 발생할 수 있는 문제를 사전에 인